In [313]:
import pandas as pd
import numpy as np
import polars as pl
df = pd.read_csv("cryptos.csv", index_col="timestamp")
period_a = 24*2
period_b = 24*3
df = df.iloc[period_a:period_b]

In [320]:
def multi_asset_hourly_strategy(df: pl.DataFrame, tickers: list[str], test_h: int, trade_h: int):
    returns_list = []
    total = 1

    df = df.sort("timestamp")

    # Ensure only relevant columns are kept
    df = df.select(["timestamp"] + tickers)

    # Convert to a pandas DataFrame for flexibility (optional)
    for start in range(0, len(df) - test_h - trade_h):
        test_window = df.slice(start, test_h)
        trade_index = start + test_h + trade_h

        best_ticker = None
        best_k = None
        best_return = -float("inf")
        for ticker in tickers:
            price = test_window[ticker]
            returns = price.pct_change().drop_nulls()
            if len(returns) < 2:
                continue
            leveraged_returns = returns.to_numpy()

            for k in range(1, 6):  # only k <= 5 allowed
                cumulative = [1]
                for i in range(1, len(leveraged_returns)):
                    if i < k:
                        strategy_ret = 0
                    else:
                        if cumulative[i - k] <= cumulative[i - 1]:
                            strategy_ret = leveraged_returns[i]
                        else:
                            strategy_ret = 0
                    cumulative.append(cumulative[i - 1] * (1 + strategy_ret))

                final_test_return = cumulative[-1]
                if final_test_return > best_return:
                    best_return = final_test_return
                    best_k = k
                    best_ticker = ticker

        if best_ticker is None or trade_index >= len(df):
            continue

        # Trade window and logic for best_ticker
        trade_slice = df.slice(trade_index - best_k, best_k)
        trade_price = trade_slice[best_ticker]
        trade_returns = trade_price.pct_change().drop_nulls().to_numpy()

        cum_growth = (trade_returns + 1).prod()

        if cum_growth <= 1:
            strategy_ret = 0
        else:
            # Get single return value at trade_index for final execution
            current_price = df[best_ticker][trade_index]
            prev_price = df[best_ticker][trade_index - 1]
            strategy_ret = (current_price / prev_price) - 1

        trade_return = 1 + strategy_ret
        total *= trade_return
        print(best_ticker, "_", best_k)
        returns_list.append({
            "start_index": start,
            "ticker": best_ticker,
            "k": best_k,
            "trade_return": trade_return,
            "total_value": total
        })

    return pd.DataFrame(returns_list)

In [323]:
tickers = ["BTCUSDT", "ETHUSDT", "XRPUSDT", "SOLUSDT", "SUIUSDT", "UNIUSDT", "AAVEUSDT"]
tickers = ["SUIUSDT"]
results = []
df_polars = pl.from_pandas(df.reset_index())  # Ensure 'timestamp' is a column
for test_h in range(1, 13):
    for trade_h in range(1, 13):
        strat_df = multi_asset_hourly_strategy(df_polars, tickers, test_h, trade_h)
        if strat_df.empty:
            total = 1.0
            final_ticker = None
            k = None
        else:
            total = strat_df["total_value"].iloc[-1]
            final_ticker = strat_df["ticker"].iloc[-1]
            k = strat_df["k"].iloc[-1]
        results.append({
            "last_ticker": final_ticker,
            "k": k,
            "test_h": test_h,
            "trade_h": trade_h,
            "total_return": round(total, 3)
        })
    print(f"Done for test_h = {test_h}...", end=" ")
results_df = pd.DataFrame(results)

Done for test_h = 1... Done for test_h = 2... SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSD

SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 5
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 1
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 5
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 5
SUIUSDT _ 4
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 5
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 2
SUIUSDT _ 3
Done for test_h = 8... SUIUSDT _ 3
SUIUSDT _ 3
SUIUSDT _ 4
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 5
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 3
SUIUSDT _ 3
SUIUSDT _ 4
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 5
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 3
SUIUSDT _ 3
SUIUSDT _ 4
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 5
SUIUSDT _ 4
SUIUSDT _ 3
SUIUSDT _ 1
SUIUSDT _ 1
SUIUSDT _ 2
SUIUSDT _ 1
SUIUSDT _ 3
SUIUSDT _ 3
SUIUSDT _ 4
SUIUS

In [324]:
results_df = results_df.sort_values(by="total_return", ascending=False).reset_index(drop=True)
results_df

,last_ticker,k,test_h,trade_h,total_return
0,SUIUSDT,1.0,7,7,1.111
1,SUIUSDT,1.0,8,6,1.111
2,SUIUSDT,1.0,6,8,1.111
3,SUIUSDT,3.0,7,9,1.106
4,SUIUSDT,3.0,6,10,1.106
...,...,...,...,...,...
139,None,NaN,2,12,1.000
140,SUIUSDT,1.0,3,2,0.999
141,SUIUSDT,1.0,4,1,0.999
142,SUIUSDT,1.0,6,2,0.998


# Unit test

In [311]:
df = pd.read_csv("cryptos.csv", index_col="timestamp")
period_a = 24*2
period_b = 24*3
df = df.iloc[period_a:period_b]
strat_df = multi_asset_hourly_strategy(pl.from_pandas(df.reset_index()), [results_df['last_ticker'][0]], int(results_df['test_h'][0]), int(results_df['trade_h'][0]))
strat_df

,start_index,ticker,k,trade_return,total_value
0,0,SUIUSDT,3,1.000000,1.000000
1,1,SUIUSDT,3,0.998890,0.998890
2,2,SUIUSDT,4,1.009419,1.008299
3,3,SUIUSDT,1,1.000000,1.008299
4,4,SUIUSDT,3,1.002236,1.010553
5,5,SUIUSDT,5,1.026676,1.037511
6,6,SUIUSDT,4,1.038531,1.077487
7,7,SUIUSDT,3,1.000085,1.077579
8,8,SUIUSDT,1,1.000000,1.077579
9,9,SUIUSDT,1,1.000000,1.077579


In [312]:
df = pd.read_csv("cryptos.csv", index_col="timestamp")
period_a = 24*2
period_b = 24*3
df = df.iloc[period_a:period_b]
strat_df = multi_asset_hourly_strategy(pl.from_pandas(df.reset_index()), [results_df['last_ticker'][0]], int(results_df['test_h'][0]), int(results_df['trade_h'][0]))
if strat_df.empty: total = 1.0
else: total = strat_df["total_value"].iloc[-1]
total

1.1010767926354292